In [12]:
import torchdata.datapipes as dp
import numpy as np
from torchvision.io import read_image
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [7]:
training_data = datasets.MNIST(
    root = 'data',
    train=True,
    download=True,
    transform=ToTensor,
)

In [8]:
test_data = datasets.MNIST(
    root='data',
    train = False,
    download=True,
    transform=ToTensor,
)

In [9]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device.")

Using cpu device.


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten
        self.linear_sigmoid_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        result = self.linear_sigmoid_stack(x)
        return result

In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_sigmoid_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Sigmoid()
  )
)


In [16]:
def train(data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss:{loss:>7f} [{current:.5d}/{size:.5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, current = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            current += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    current /= size
    print(f"Test Error: \n Accuracy: {(100 * current):>0.1f}%, Avg loss: {test_loss:>8f}\n")